# PyEO Forest Alerts: How to detect land cover changes between a time-series of Sentinel-2 images and a baseline image composite from the past using a trained machine learning classification model.

This notebook was developed for pyeo on a Linux VM for Azure Labs.

- This notebook will cover how to query the Sentinel-2 image archive on the Copernicus Data Space Ecosystem (CDSE) to download selected images, and how to download a set of Sentinel-2 images from which we can detect changes in land cover since the baseline image composite.
- Only changes from a subset of forest classes to non-forest classes will be identified as a candidate forest alert.
- This will be done by classifying the baseline composite and each change detection image using the same machine learning model.
- The machine learning model is checked against a change in NDVI over time to increase the confidence in a true forest alert.

## Downloading and pre-processing a time series of Sentinel-2 images for change detection

- This section will take us stepwise through the imagery query, download and cloud-masking aspects of the `run_acd_national.py` script, which runs the full PyEO pipeline from the command line in a terminal.  
- Jupyter notebooks provide a useful and engaging interface to understand the components of this script, so we will follow an extracted version throughout this notebook.

This section comprises several stages:   
1. Directory and Variable setup.
1. Querying for Sentinel-2 imagery that meets our search criteria.
1. Downloading the Sentinel-2 imagery identified from the Query.
1. If necessary, preprocess any L1C to L2A by applying atmospheric corrections.
1. Cloud-masking the L2A imagery.
1. Classification of all Change Detection Images
1. Creation of Forest Alerts

## Setup: Requirements to use this Notebook

In [5]:
import os
pyeo_dir = '/home/cmsstudent/pyeo'
os.chdir(pyeo_dir)
workdir = os.getcwd()
print(workdir)
config_path = os.path.join(pyeo_dir, 'pyeo_linux_azure.ini')


/home/cmsstudent/pyeo


We did this in the previous notebook step-by-step. Here, we initialie the notebook in one code cell to speed up the process.

In [7]:
%%time
import shutil
import sys
import configparser
import argparse
import json
import numpy as np
from osgeo import gdal
import geopandas as gpd
import glob
import pandas as pd
from datetime import datetime
import warnings
import zipfile

from pyeo import (
    classification,
    filesystem_utilities,
    queries_and_downloads,
    raster_manipulation,
    vectorisation
)

from pyeo.filesystem_utilities import (
    initialisation,
    config_to_log,
    move_file,
    move_and_rename_old_file
)

from pyeo.raster_manipulation import roi_tile_intersection

gdal.UseExceptions()
warnings.simplefilter("ignore", category=UserWarning)

# navigate into the pyeo directory
os.chdir(pyeo_dir)
#----------------------------------------------------------
# read in the ini file into a dictionary and start the log file
config_dict, my_log = initialisation(config_path)
#----------------------------------------------------------
# print the contents of the ini file to the log file
config_to_log(config_dict, my_log)
#----------------------------------------------------------
# get the tile IDs that intersect with the ROI (region of interest)
tilelist_filepath = roi_tile_intersection(config_dict, my_log)
my_log.info(tilelist_filepath)
#----------------------------------------------------------
# Select only the first tile for the purposes of this notebook
tile_to_process = pd.read_csv(tilelist_filepath)["tile"][0]
# create a subdirectory for that tile
individual_tile_directory_path = os.path.join(
    config_dict["tile_dir"],
    tile_to_process
    )
# create all necessary subdirectories there
filesystem_utilities.create_folder_structure_for_tiles(
    individual_tile_directory_path
    )
#----------------------------------------------------------
# initialise the tile_log file for that tile
tile_log = filesystem_utilities.init_log_acd(
    log_path=os.path.join(
        individual_tile_directory_path,
        "log",
        tile_to_process + ".log"
        ),
    logger_name=f"pyeo_{tile_to_process}"
)
#----------------------------------------------------------
# assign some contents of the ini file dictionary to simpler variables
start_date = config_dict["start_date"]
# convert the TODAY flag if specified into the current date and time stamp
if start_date == "TODAY":
  start_date = datetime.today().strftime('%Y%m%d')
end_date = config_dict["end_date"]
if end_date == "TODAY":
  end_date = datetime.today().strftime('%Y%m%d')
composite_start_date = config_dict["composite_start"]
composite_end_date = config_dict["composite_end"]
cloud_cover = config_dict["cloud_cover"]
cloud_certainty_threshold = config_dict["cloud_certainty_threshold"]
model_path = config_dict["model_path"]
sen2cor_path = config_dict["sen2cor_path"]
epsg = config_dict["epsg"]
bands = config_dict["bands"]
resolution = config_dict["resolution_string"]
out_resolution = config_dict["output_resolution"]
buffer_size = config_dict["buffer_size_cloud_masking"]
buffer_size_composite = config_dict["buffer_size_cloud_masking_composite"]
download_limit = config_dict["download_limit"]
faulty_granule_threshold = config_dict["faulty_granule_threshold"]
skip_existing = config_dict["do_skip_existing"]
sieve = config_dict["sieve"]
from_classes = config_dict["from_classes"]
to_classes = config_dict["to_classes"]

download_source = config_dict["download_source"]
if download_source == "scihub":
    tile_log.info("scihub API is the download source")
if download_source == "dataspace":
    tile_log.info("dataspace API is the download source")

credentials_path = config_dict["credentials_path"]
if not os.path.exists(credentials_path):
    tile_log.error(f"The credentials path does not exist  :{credentials_path}")
    tile_log.error(f"Current working directory :{os.getcwd()}")
    tile_log.error("Exiting raster pipeline")
    sys.exit(1)
conf = configparser.ConfigParser(allow_no_value=True, interpolation=None)
conf.read(credentials_path)
credentials_dict = {}
#----------------------------------------------------------------------
# define variables that point to the directory paths
change_image_dir = os.path.join(individual_tile_directory_path, r"images")
l1_image_dir = os.path.join(individual_tile_directory_path, r"images", r"L1C")
l2_image_dir = os.path.join(individual_tile_directory_path, r"images", r"L2A")
l2_masked_image_dir = os.path.join(individual_tile_directory_path, r"images", r"cloud_masked")
categorised_image_dir = os.path.join(individual_tile_directory_path, r"output", r"classifications")
probability_image_dir = os.path.join(individual_tile_directory_path, r"output", r"probabilities")
report_image_dir = os.path.join(individual_tile_directory_path, r"output", r"reports")
sieved_image_dir = os.path.join(individual_tile_directory_path, r"output", r"sieved")
composite_dir = os.path.join(individual_tile_directory_path, r"composite")
quicklook_dir = os.path.join(individual_tile_directory_path, r"output", r"quicklooks")
#------------------------------------------------------------------------
if download_source == "dataspace":
    tile_log.info(f'Running download handler for {download_source}')
    credentials_dict["sent_2"] = {}
    credentials_dict["sent_2"]["user"] = conf["dataspace"]["user"]
    credentials_dict["sent_2"]["pass"] = conf["dataspace"]["pass"]
    sen_user = credentials_dict["sent_2"]["user"]
    sen_pass = credentials_dict["sent_2"]["pass"]
if download_source == "scihub":
    tile_log.info(f'Running download handler for {download_source}')
    credentials_dict["sent_2"] = {}
    credentials_dict["sent_2"]["user"] = conf["sent_2"]["user"]
    credentials_dict["sent_2"]["pass"] = conf["sent_2"]["pass"]
    sen_user = credentials_dict["sent_2"]["user"]
    sen_pass = credentials_dict["sent_2"]["pass"]

2024-09-25 16:14:50,710: INFO: ---------------------------------------------------------------
2024-09-25 16:14:50,711: INFO: ---                 PROCESSING START                        ---
2024-09-25 16:14:50,712: INFO: ---------------------------------------------------------------
2024-09-25 16:14:50,712: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-09-25 16:14:50,713: INFO: True
2024-09-25 16:14:50,713: INFO: Reading in parameters defined in: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-09-25 16:14:50,714: INFO: ---------------------------------------------------------------
2024-09-25 16:14:50,714: INFO: ----------------------------
2024-09-25 16:14:50,715: INFO: Contents of the config file:
2024-09-25 16:14:50,716: INFO: ----------------------------
2024-09-25 16:14:50,716: INFO:   run_mode :  watch_period_seconds
2024-09-25 16:14:50,717: INFO:   forest_sentinel :  model
2024-09-25 16:14:50,717: INFO:   environment :  sen2cor_path
2024-09

CPU times: user 148 ms, sys: 28.1 ms, total: 176 ms
Wall time: 159 ms


# Download Sentinel-2 change detection images

In [8]:
%%time
tile_log.info(tile_to_process)
if config_dict["do_all"] or config_dict["do_download"]:
    tile_log.info("---------------------------------------------------------------")
    tile_log.info(f"Downloading change detection images between {start_date} and "+
        f"{end_date} with cloud cover <= {cloud_cover}")
    tile_log.info("---------------------------------------------------------------")
    if download_source == "dataspace":
        try:
            tiles_geom_path = os.path.join(
                pyeo_dir,
                config_dict["geometry_dir"],
                config_dict["s2_tiles_filename"]
                )
            tiles_geom = gpd.read_file(os.path.abspath(tiles_geom_path))
        except FileNotFoundError:
            tile_log.error(f"tiles_geom does not exist, the path is :{tiles_geom_path}")

        tile_geom = tiles_geom[tiles_geom["Name"] == tile_to_process]
        tile_geom = tile_geom.to_crs(epsg=4326)
        geometry = tile_geom["geometry"].iloc[0]
        geometry = geometry.representative_point()

        # convert date string to YYYY-MM-DD
        date_object = datetime.strptime(start_date, "%Y%m%d")
        dataspace_change_start = date_object.strftime("%Y-%m-%d")
        date_object = datetime.strptime(end_date, "%Y%m%d")
        dataspace_change_end = date_object.strftime("%Y-%m-%d")

        try:
            dataspace_change_products_all = queries_and_downloads.query_dataspace_by_tile_id(
                max_cloud_cover=cloud_cover,
                start_date=dataspace_change_start,
                end_date=dataspace_change_end,
                tile_id = tile_to_process,
                max_records=100,
                log=tile_log
            )
        except Exception as error:
            tile_log.error(f"query_dataspace_by_tile_id received this error: {error}")

        titles = dataspace_change_products_all["title"].tolist()
        sizes = list()
        uuids = list()
        for elem in dataspace_change_products_all.itertuples(index=False):
            sizes.append(elem[-2]["download"]["size"])
            uuids.append(elem[-2]["download"]["url"].split("/")[-1])

        relative_orbit_numbers = dataspace_change_products_all["relativeOrbitNumber"].tolist()
        processing_levels = dataspace_change_products_all["processingLevel"].tolist()
        transformed_levels = ['Level-1C' if level == 'S2MSI1C' else 'Level-2A' for level in processing_levels]
        cloud_covers = dataspace_change_products_all["cloudCover"].tolist()
        begin_positions = dataspace_change_products_all["startDate"].tolist()
        statuses = dataspace_change_products_all["status"].tolist()

        scihub_compatible_df = pd.DataFrame({"title": titles,
                                            "size": sizes,
                                            "beginposition": begin_positions,
                                            "relativeorbitnumber": relative_orbit_numbers,
                                            "cloudcoverpercentage": cloud_covers,
                                            "processinglevel": transformed_levels,
                                            "uuid": uuids,
                                            "status": statuses})

        # check granule sizes on the server
        scihub_compatible_df["size"] = scihub_compatible_df["size"].apply(lambda x: round(float(x) * 1e-6, 2))

        # reassign to match the scihub variable
        df_all = scihub_compatible_df

    if download_source == "scihub":
        products_all = queries_and_downloads.check_for_s2_data_by_date(
            config_dict["tile_dir"],
            start_date,
            end_date,
            credentials_dict,
            cloud_cover=cloud_cover,
            tile_id=tile_to_process,
            producttype=None,  # "S2MSI2A" or "S2MSI1C"
        )
        tile_log.info(
            "--> Found {} L1C and L2A products for change detection:".format(
                len(products_all)
            )
        )
        df_all = pd.DataFrame.from_dict(products_all, orient="index")

        # check granule sizes on the server
        df_all["size"] = (
            df_all["size"]
            .str.split(" ")
            .apply(lambda x: float(x[0]) * {"GB": 1e3, "MB": 1, "KB": 1e-3}[x[1]])
        )

    # here the main call (from if download_source == "scihub" branch) is resumed
    df = df_all.query("size >= " + str(faulty_granule_threshold))
    tile_log.info(
        "Removed {} faulty scenes <{}MB in size from the list:".format(
            len(df_all) - len(df), faulty_granule_threshold
        )
    )
    df_faulty = df_all.query("size < " + str(faulty_granule_threshold))
    for r in range(len(df_faulty)):
        tile_log.info(
            "   {} MB: {}".format(
                df_faulty.iloc[r, :]["size"], df_faulty.iloc[r, :]["title"]
            )
        )

    l1c_products = df[df.processinglevel == "Level-1C"]
    l2a_products = df[df.processinglevel == "Level-2A"]
    tile_log.info("    {} L1C products".format(l1c_products.shape[0]))
    tile_log.info("    {} L2A products".format(l2a_products.shape[0]))

    if l1c_products.shape[0] > 0 and l2a_products.shape[0] > 0:
        tile_log.info(
            "Filtering out L1C products that have the same 'beginposition' time "+
            "stamp as an existing L2A product."
            )
        if download_source == "scihub":
            (l1c_products,l2a_products,) = \
                queries_and_downloads.filter_unique_l1c_and_l2a_data(
                    df,
                    log=tile_log
                    )

        if download_source == "dataspace":
            l1c_products = \
                queries_and_downloads.filter_unique_dataspace_products(
                    l1c_products=l1c_products,
                    l2a_products=l2a_products,
                    log=tile_log
                    )

        n = l1c_products.shape[0] + l2a_products.shape[0]
        tile_log.info(f"--> {n} L1C and L2A products with unique 'beginposition' "+
            "time stamp:")

    df = None
    tile_log.info(f" {len(l1c_products['title'])} L1C Change Images")
    tile_log.info(f" {len(l2a_products['title'])} L2A Change Images")

    tile_log.info("Cell successfully finished")

2024-09-25 16:15:03,523: INFO: 36NXG
2024-09-25 16:15:03,524: INFO: ---------------------------------------------------------------
2024-09-25 16:15:03,525: INFO: Downloading change detection images between 20240201 and 20240925 with cloud cover <= 25
2024-09-25 16:15:03,526: INFO: ---------------------------------------------------------------
2024-09-25 16:15:05,374: INFO: Removed 3 faulty scenes <200MB in size from the list:
2024-09-25 16:15:05,378: INFO:    199.97 MB: S2B_MSIL1C_20240223T075929_N0510_R035_T36NXG_20240223T095005.SAFE
2024-09-25 16:15:05,379: INFO:    197.15 MB: S2B_MSIL1C_20240602T075609_N0510_R035_T36NXG_20240602T094607.SAFE
2024-09-25 16:15:05,380: INFO:    198.23 MB: S2B_MSIL1C_20240702T075609_N0510_R035_T36NXG_20240702T094619.SAFE
2024-09-25 16:15:05,382: INFO:     54 L1C products
2024-09-25 16:15:05,383: INFO:     37 L2A products
2024-09-25 16:15:05,383: INFO: Filtering out L1C products that have the same 'beginposition' time stamp as an existing L2A product.
2

CPU times: user 212 ms, sys: 29.1 ms, total: 241 ms
Wall time: 1.99 s


## Search for L2A Images Corresponding to L1C

In [9]:
%%time
if config_dict["do_all"] or config_dict["do_download"]:
    if download_source == "scihub":
        if l1c_products.shape[0] > 0:
            tile_log.info("Checking for availability of L2A products to minimise download and atmospheric correction of L1C products.")
            n = len(l1c_products)
            drop = []
            add = []
            for r in range(n):
                id = l1c_products.iloc[r, :]["title"]
                search_term = (
                    "*"
                    + id.split("_")[2]
                    + "_"
                    + id.split("_")[3]
                    + "_"
                    + id.split("_")[4]
                    + "_"
                    + id.split("_")[5]
                    + "*"
                )
                tile_log.info("Search term: {}.".format(search_term))
                matching_l2a_products = queries_and_downloads._file_api_query(
                    user=sen_user,
                    passwd=sen_pass,
                    start_date=start_date,
                    end_date=end_date,
                    filename=search_term,
                    cloud=cloud_cover,
                    producttype="S2MSI2A",
                )

                matching_l2a_products_df = pd.DataFrame.from_dict(
                    matching_l2a_products, orient="index"
                )
                if len(matching_l2a_products_df) == 1:
                    tile_log.info(matching_l2a_products_df.iloc[0, :]["size"])
                    matching_l2a_products_df["size"] = (
                        matching_l2a_products_df["size"]
                        .str.split(" ")
                        .apply(
                            lambda x: float(x[0])
                            * {"GB": 1e3, "MB": 1, "KB": 1e-3}[x[1]]
                        )
                    )
                    if (
                        matching_l2a_products_df.iloc[0, :]["size"]
                        > faulty_granule_threshold
                    ):
                        tile_log.info("Replacing L1C {} with L2A product:".format(id))
                        tile_log.info(
                            "              {}".format(
                                matching_l2a_products_df.iloc[0, :]["title"]
                            )
                        )
                        drop.append(l1c_products.index[r])
                        add.append(matching_l2a_products_df.iloc[0, :])
                if len(matching_l2a_products_df) == 0:
                    tile_log.info("Found no match for L1C: {}.".format(id))
                if len(matching_l2a_products_df) > 1:
                    # check granule sizes on the server
                    matching_l2a_products_df["size"] = (
                        matching_l2a_products_df["size"]
                        .str.split(" ")
                        .apply(
                            lambda x: float(x[0])
                            * {"GB": 1e3, "MB": 1, "KB": 1e-3}[x[1]]
                        )
                    )
                    if (
                        matching_l2a_products_df.iloc[0, :]["size"]
                        > faulty_granule_threshold
                    ):
                        tile_log.info("Replacing L1C {} with L2A product:".format(id))
                        tile_log.info(
                            "              {}".format(
                                matching_l2a_products_df.iloc[0, :]["title"]
                            )
                        )
                        drop.append(l1c_products.index[r])
                        add.append(matching_l2a_products_df.iloc[0, :])

            if len(drop) > 0:
                l1c_products = l1c_products.drop(index=drop)
            if len(add) > 0:
                if config_dict["do_dev"]:
                    add = pd.DataFrame(add)
                    l2a_products = pd.concat([l2a_products, add])
                else:
                    add = pd.DataFrame(add)
                    l2a_products = pd.concat([l2a_products, add])

            tile_log.info(
                "    {} L1C products remaining for download".format(
                    l1c_products.shape[0]
                )
            )
    l2a_products = l2a_products.drop_duplicates(subset="title")
    tile_log.info("    {} L2A products remaining for download".format(l2a_products.shape[0]))

2024-09-25 16:15:20,887: INFO:     37 L2A products remaining for download


CPU times: user 0 ns, sys: 2.71 ms, total: 2.71 ms
Wall time: 2.29 ms


## Download and Pre-Process L1C Change Imagery

- If there any L1C products in the change images search query that are not matched with L2A products, then download these L1Cs and apply `atmospheric_correction`.

In [10]:
%%time
if config_dict["do_all"] or config_dict["do_download"]:
    if l1c_products.shape[0] > 0:
        # only download the L1C products if we have sen2cor installed to
        #   apply the atmospheric correction to L2A level
        if os.path.exists(config_dict['sen2cor_path']):
            tile_log.info("  Sen2Cor path found.")

            tile_log.info(f"Downloading Sentinel-2 L1C products from {download_source}")

            if download_source == "scihub":
                queries_and_downloads.download_s2_data_from_df(
                    l1c_products,
                    l1_image_dir,
                    l2_image_dir,
                    download_source,
                    user=sen_user,
                    passwd=sen_pass,
                    try_scihub_on_fail=True,
                )
            elif download_source == "dataspace":
                    queries_and_downloads.download_s2_data_from_dataspace(
                    product_df=l1c_products,
                    l1c_directory=l1_image_dir,
                    l2a_directory=l2_image_dir,
                    dataspace_username=sen_user,
                    dataspace_password=sen_pass,
                    log=tile_log
                )
            else:
                tile_log.error(
                    f"download source specified did not match 'scihub' or 'dataspace'"
                    )
                tile_log.error(
                    f"download source supplied was  :  {download_source}"
                    )
                tile_log.error("Exiting pipeline...")
                sys.exit(1)

            tile_log.info("Downloaded the Sentinel-2 L1C products.")
            tile_log.info("Atmospheric correction with sen2cor.")
            raster_manipulation.atmospheric_correction(
                l1_image_dir,
                l2_image_dir,
                sen2cor_path,
                delete_unprocessed_image=False,
                log=tile_log,
            )

        else:
            tile_log.warning("  Sen2Cor path does not exist. Cannot convert L1C "+
                        "to L2A. Skipping download of L1C images.")

2024-09-25 16:15:28,608: WARNING:   Sen2Cor path does not exist. Cannot convert L1C to L2A. Skipping download of L1C images.


CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.58 ms


## Download L2A Change Imagery

In [11]:
%%time
if config_dict["do_all"] or config_dict["do_download"]:
    if l2a_products.shape[0] > download_limit:
       tile_log.warning(f"L2A image download list contains {l2a_products.shape[0]}"+
                        " L2A products. This exceeds the set download limit "+
                        f"of {download_limit}. Only the first images will be downloaded.")
       l2a_products = l2a_products[0:download_limit]
    if l2a_products.shape[0] > 0:
        tile_log.info(f"Downloading Sentinel-2 L2A products from {download_source}")

        if download_source == "scihub":
            queries_and_downloads.download_s2_data(
                l2a_products.to_dict("index"),
                l1_image_dir,
                l2_image_dir,
                download_source,
                user=sen_user,
                passwd=sen_pass,
                try_scihub_on_fail=True,
            )
        if download_source == "dataspace":
            queries_and_downloads.download_s2_data_from_dataspace(
                product_df=l2a_products,
                l1c_directory=l1_image_dir,
                l2a_directory=l2_image_dir,
                dataspace_username=sen_user,
                dataspace_password=sen_pass,
                log=tile_log
            )

    # check for incomplete L2A downloads and remove them
    incomplete_downloads, sizes = raster_manipulation.find_small_safe_dirs(
        l2_image_dir, threshold=faulty_granule_threshold * 1024 * 1024
    )
    if len(incomplete_downloads) > 0:
        for index, safe_dir in enumerate(incomplete_downloads):
            if sizes[
                index
            ] / 1024 / 1024 < faulty_granule_threshold and os.path.exists(safe_dir):
                tile_log.warning(
                    "Found likely incomplete download of size {} MB: {}".format(
                        str(round(sizes[index] / 1024 / 1024)), safe_dir
                    )
                )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info("L2A image download of change detection images is complete.")
    tile_log.info("---------------------------------------------------------------")

2024-09-25 16:15:34,461: WARNING: L2A image download list contains 37 L2A products. This exceeds the set download limit of 3. Only the first images will be downloaded.
2024-09-25 16:15:34,463: INFO: Downloading Sentinel-2 L2A products from dataspace
2024-09-25 16:15:34,465: INFO: --------------------------------------------------------------------------------
2024-09-25 16:15:34,465: INFO: Checking 1 of 3 : S2B_MSIL2A_20240220T074949_N0510_R135_T36NXG_20240220T100534.SAFE
2024-09-25 16:15:34,467: INFO: /home/cmsstudent/Desktop/pyeo_data/36NXG/images/L2A/S2B_MSIL2A_20240220T074949_N0510_R135_T36NXG_20240220T100534.SAFE does not exist.
2024-09-25 16:15:34,468: INFO:     Downloading  : S2B_MSIL2A_20240220T074949_N0510_R135_T36NXG_20240220T100534.SAFE
2024-09-25 16:15:35,128: INFO: response.status_code: 301
2024-09-25 16:15:35,129: INFO: download url = response.headers['Location']: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(f3b18418-8a26-4059-b833-1a74eab07dbd)/$value
2024

CPU times: user 21.9 s, sys: 16.2 s, total: 38.2 s
Wall time: 14min 3s


## Housekeeping - Compress L1Cs

If you have set your `do_zip` argument to `True`, then this cell will compress the L1Cs now that they have been atmospherically corrected and relabelled as L2As.

In [12]:
%%time
if config_dict["do_all"] or config_dict["do_download"]:
    if config_dict["do_delete"]:
        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Deleting L1C images downloaded for change detection.")
        tile_log.info("Keeping only the derived L2A images after atmospheric correction.")
        tile_log.info("---------------------------------------------------------------")
        directory = l1_image_dir
        tile_log.info("Deleting {}".format(directory))
        shutil.rmtree(directory)
        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Deletion complete")
        tile_log.info("---------------------------------------------------------------")
    else:
        if config_dict["do_zip"]:
            tile_log.info("---------------------------------------------------------------")
            tile_log.info("Zipping L1C images downloaded for change detection")
            tile_log.info("---------------------------------------------------------------")
            filesystem_utilities.zip_contents(l1_image_dir)
            tile_log.info("---------------------------------------------------------------")
            tile_log.info("Zipping complete")
            tile_log.info("---------------------------------------------------------------")

    tile_log.info("Housekeeping cell successfully finished")

2024-09-25 16:29:37,778: INFO: ---------------------------------------------------------------
2024-09-25 16:29:37,780: INFO: Zipping L1C images downloaded for change detection
2024-09-25 16:29:37,780: INFO: ---------------------------------------------------------------
2024-09-25 16:29:38,505: INFO: ---------------------------------------------------------------
2024-09-25 16:29:38,506: INFO: Zipping complete
2024-09-25 16:29:38,507: INFO: ---------------------------------------------------------------
2024-09-25 16:29:38,508: INFO: Housekeeping cell successfully finished


CPU times: user 5.83 ms, sys: 2.59 ms, total: 8.42 ms
Wall time: 730 ms


# Cloud Masking, Offsetting and Quicklooks

Here, like before in the previous session, we cloud mask, apply the baseline offset correction and produce quicklooks (if selected).  

Additionally, if you have set the `do_zip` flag to True, then `pyeo` will compress the cloud masked L2A images, as we no longer need these once classified.

## Cloud Masking

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_change"]:
    tile_log.info("---------------------------------------------------------------")
    tile_log.info(
        "Applying simple cloud, cloud shadow and haze mask based on SCL files "+
        "and stacking the masked band raster files."
    )
    tile_log.info("---------------------------------------------------------------")

    directory = l2_masked_image_dir
    masked_file_paths = [
        f
        for f in os.listdir(directory)
        if f.endswith(".tif") and os.path.isfile(os.path.join(directory, f))
    ]

    # unzip any zip files in the l2_image_dir if they have not been cloud masked yet
    directory = l2_image_dir
    l2a_zip_file_paths = [f for f in os.listdir(directory) if f.endswith(".zip")]
    if len(l2a_zip_file_paths) > 0:
        for f in l2a_zip_file_paths:
            # check whether the zipped file has already been cloud masked
            zip_timestamp = filesystem_utilities.get_image_acquisition_time(
                os.path.basename(f)
            ).strftime("%Y%m%dT%H%M%S")
            if any(zip_timestamp in f for f in masked_file_paths):
                continue
            else:
                # extract it if not
                filesystem_utilities.unzip_contents(
                    os.path.join(directory, f),
                    ifstartswith="S2",
                    ending=".SAFE",
                )

    # get list of .SAFE files for cloud masking after unzipping
    l2a_safe_file_paths = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.endswith(".SAFE") and os.path.isdir(os.path.join(directory, f))
    ]

    files_for_cloud_masking = []
    if len(l2a_safe_file_paths) > 0:
        for f in l2a_safe_file_paths:
            # check whether the L2A SAFE file has already been cloud masked
            safe_timestamp = filesystem_utilities.get_image_acquisition_time(
                os.path.basename(f)
            ).strftime("%Y%m%dT%H%M%S")
            if any(safe_timestamp in f for f in masked_file_paths):
                continue
            else:
                # add it to the list of files to do if it has not been cloud masked yet
                files_for_cloud_masking = files_for_cloud_masking + [f]

    if len(files_for_cloud_masking) == 0:
        tile_log.info("No L2A images found for cloud masking. They may already have been done.")
    else:
        tile_log.info("Files for cloud masking:")
        for f in files_for_cloud_masking:
            tile_log.info(f"  {f}")
        raster_manipulation.apply_scl_cloud_mask_to_filelist(
            files_for_cloud_masking,
            l2_masked_image_dir,
            scl_classes=[0, 1, 2, 3, 8, 9, 10, 11],
            buffer_size=buffer_size,
            bands=bands,
            out_resolution=out_resolution,
            haze=None,
            epsg=epsg,
            skip_existing=skip_existing,
            log=tile_log
        )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Cloud masking and band stacking of new L2A images are complete.")
    tile_log.info("---------------------------------------------------------------")

2024-09-25 16:29:39,497: INFO: ---------------------------------------------------------------
2024-09-25 16:29:39,498: INFO: Applying simple cloud, cloud shadow and haze mask based on SCL files and stacking the masked band raster files.
2024-09-25 16:29:39,499: INFO: ---------------------------------------------------------------
2024-09-25 16:29:39,502: INFO: Files for cloud masking:
2024-09-25 16:29:39,502: INFO:   /home/cmsstudent/Desktop/pyeo_data/36NXG/images/L2A/S2B_MSIL2A_20240314T075709_N0510_R035_T36NXG_20240314T101645.SAFE
2024-09-25 16:29:39,503: INFO:   /home/cmsstudent/Desktop/pyeo_data/36NXG/images/L2A/S2B_MSIL2A_20240220T074949_N0510_R135_T36NXG_20240220T100534.SAFE
2024-09-25 16:29:39,504: INFO:   /home/cmsstudent/Desktop/pyeo_data/36NXG/images/L2A/S2A_MSIL2A_20240316T074651_N0510_R135_T36NXG_20240316T110248.SAFE
2024-09-25 16:29:39,504: INFO: 3 L2A raster files are in the file list for SCL cloud masking.
2024-09-25 16:29:39,505: INFO:   Applying SCL cloud mask to L2A 

## Radiometric Offsetting

Let's correct for the radiometric offset to the digital numbers in the Sentinel-2 images that were processed with processing baseline 4.00 or later.
https://sentiwiki.copernicus.eu/web/s2-processing


In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_change"]:
    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Offsetting cloud masked L2A images.")
    tile_log.info("---------------------------------------------------------------")

    raster_manipulation.apply_processing_baseline_offset_correction_to_tiff_file_directory(
        in_tif_directory = l2_masked_image_dir,
        out_tif_directory = l2_masked_image_dir,
        bands_to_offset_labels=("B02", "B03", "B04", "B08"),
        bands_to_offset_index=[0, 1, 2, 3],
        BOA_ADD_OFFSET=-1000,
        backup_flag=False,
        log = tile_log
    )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Offsetting of cloud masked L2A images complete.")
    tile_log.info("---------------------------------------------------------------")

## Making Quicklooks

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_download"]:
    if config_dict["do_quicklooks"] or config_dict["do_all"]:
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info("Producing quicklooks.")
        tile_log.info(
            "---------------------------------------------------------------"
        )
        dirs_for_quicklooks = [l2_masked_image_dir, l2_image_dir]
        for main_dir in dirs_for_quicklooks:
            files = [
                f.path
                for f in os.scandir(main_dir)
                if f.is_file() and os.path.basename(f).endswith(".tif")
            ]
            if len(files) == 0:
                tile_log.warning("No images found in {}.".format(main_dir))
            else:
                for f in files:
                    quicklook_path = os.path.join(
                        quicklook_dir,
                        os.path.basename(f).split(".")[0] + ".png",
                    )
                    tile_log.info("Creating quicklook: {}".format(quicklook_path))
                    raster_manipulation.create_quicklook(
                        f,
                        quicklook_path,
                        width=512,
                        height=512,
                        format="PNG",
                        bands=[3, 2, 1],
                        nodata=0,
                        scale_factors=[[0, 2000, 0, 255]],
                        log=tile_log,
                    )
        tile_log.info("Quicklooks complete.")
    else:
        tile_log.info("Quicklooks disabled in ini file.")


## Housekeeping

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_change"]:
    if config_dict["do_zip"]:
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info("Zipping L2A images downloaded for change detection")
        tile_log.info(
            "---------------------------------------------------------------"
        )
        filesystem_utilities.zip_contents(l2_image_dir)
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info("Zipping complete")
        tile_log.info(
            "---------------------------------------------------------------"
        )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info(
        "Compressing tiff files in directory {} and all subdirectories".format(
            l2_masked_image_dir
        )
    )
    tile_log.info("---------------------------------------------------------------")
    for root, dirs, files in os.walk(l2_masked_image_dir):
        all_tiffs = [
            image_name for image_name in files if image_name.endswith(".tif")
        ]
        for this_tiff in all_tiffs:
            raster_manipulation.compress_tiff(
                os.path.join(root, this_tiff),
                os.path.join(root, this_tiff),
                log=tile_log
            )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info(
        "Pre-processing of change detection images, file compression, zipping"
    )
    tile_log.info(
        "and deletion of intermediate file products (if selected) are complete."
    )
    tile_log.info("---------------------------------------------------------------")



# Classification of the Baseline Composite and Change Images

In [ ]:
# get all file names from composite_dir that start with "composite"
composite_paths = [
    os.path.join(composite_dir, f)
    for f in os.listdir(composite_dir)
    if f.startswith("composite") and f.endswith(".tif") and
    os.path.isfile(os.path.join(composite_dir, f))
]

if len(composite_paths) == 0:
    tile_log.warning("No composite images found in {}".format(composite_dir))
else:
    tile_log.info("Found image composite files:")
    for f in composite_paths:
        tile_log.info("  {}".format(f))

## Run the classification and apply the machine learning model

Here, we classify the Baseline Composite and the Change images using the model we created in the model training session.

In [ ]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    if config_dict["do_all"] or config_dict["do_classify"]:
        tile_log.info("---------------------------------------------------------------")
        tile_log.info(
            "Classify a land cover map for each L2A change detection image and "+
            "composite image using a saved model"
        )
        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Model used: {}".format(model_path))

        if skip_existing:
            tile_log.info("Skipping existing classification images if found.")

        classification.classify_directory(
            composite_dir,
            model_path,
            categorised_image_dir,
            prob_out_dir=None,
            apply_mask=False,
            out_type="GTiff",
            chunks=config_dict["chunks"],
            skip_existing=skip_existing,
            log=tile_log
        )

        classification.classify_directory(
            l2_masked_image_dir,
            model_path,
            categorised_image_dir,
            prob_out_dir=None,
            apply_mask=False,
            out_type="GTiff",
            chunks=config_dict["chunks"],
            skip_existing=skip_existing,
            log=tile_log
        )

        tile_log.info("---------------------------------------------------------------")
        tile_log.info(
            f"LZW-compressing tiff files in directory {categorised_image_dir} "+
            "and all subdirectories"
            )
        tile_log.info("---------------------------------------------------------------")

        for root, dirs, files in os.walk(categorised_image_dir):
            all_tiffs = [
                image_name for image_name in files if image_name.endswith(".tif")
            ]
            for this_tiff in all_tiffs:
                raster_manipulation.compress_tiff(
                    os.path.join(root, this_tiff),
                    os.path.join(root, this_tiff),
                    tile_log
                )

        tile_log.info(
            "---------------------------------------------------------------"
        )

        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Classification of all images is complete.")
        tile_log.info("---------------------------------------------------------------")


## Housekeeping

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_classify"]:
    if config_dict["do_quicklooks"] or config_dict["do_all"]:
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info("Producing quicklooks.")
        tile_log.info(
            "---------------------------------------------------------------"
        )

        # do classification images only
        dirs_for_quicklooks = [categorised_image_dir]

        for main_dir in dirs_for_quicklooks:
            files = [
                os.path.join(main_dir, f)
                for f in os.listdir(main_dir)
                if os.path.isfile(os.path.join(main_dir, f)) and
                f.endswith("class.tif")
            ]
            if len(files) == 0:
                tile_log.warning("No images found in {}.".format(main_dir))
            else:
                for f in files:
                    quicklook_path = os.path.join(
                        quicklook_dir,
                        os.path.basename(f).split(".")[0] + ".png",
                    )
                    tile_log.info("Creating quicklook: {}".format(quicklook_path))
                    raster_manipulation.create_quicklook(
                        in_raster_path = f,
                        out_raster_path = quicklook_path,
                        width=512,
                        height=512,
                        format="PNG",
                        bands=[1],
                        nodata=None,
                        scale_factors=None,
                        log=tile_log
                    )
        tile_log.info("Quicklooks complete.")
    else:
        tile_log.info("Quicklooks disabled in ini file.")


# Post-classification Change Detection

To perform Change Detection, we take the Classified Change Imagery and compare it with the Classified Baseline Composite.

Because we are concerned with monitoring deforestation for our Change Detection, `pyeo` examines whether any forest classes (*classes 1, 11 and 12*) change to non-forest classes (*classes 3, 4, 5 and 13*).  

As new change imagery becomes available (*as deforestation monitoring is an iterative process through time*), these change images are classified and compared to the baseline, again.

---

## Detect change between a new stack of images and the baseline composite

detect_change.pyis an application that downloads and preprocesses a stack of Sentinel-2 images for change detection and classifies them using a machine learning model. It performs change detection against a baseline median image composite. The application then generates a report image file and optionally vectorises it if selected in the ini file.

It can be run in two basic ways:

1. detect_change.py ini_file_path.ini
In this configuration, the composite creation is based on the processing parameters defined in the ini file.

2. detect_change.py ini_file_path.ini --tile $tile_id
The --tile flag identifies a tile ID of the Sentinel-2 tile. Specifying a tile ID overrides the geojson location of the area of interest from the ini file with a Sentinel-2 tile ID location.


The overall Change Detection can be summarised as this:
- PyEO first looks for the composite and change imagery classifications and orders them by most recent.
- Then, it searches for existing report files created from previous PyEO runs and archive them, moving them to an archived folder.
- Then it creates the change report by sequentially comparing the classified change imagery against the classified baseline composite.
- Once finished, PyEO does some housekeeping, compressing unneeded files.



## Meaning of the report file layers (bands), starting numbering from 0:
Layer 0: Total Image Count: Counts the number of images processed per pixel - number of available images within overall cloud percentage cover limit set in pyeo.ini file.

Layer 1: Occluded Image Count: Counts number of occasions a pixel is obscured by cloud or out-of-orbit

Layer 2: Classifier Change Detection Count: Count if a from/to change of land cover classes was detected

Layer 3: First-Change Trigger for Combined Classifier + dNDVI Hybrid Change Detection: Records the earliest date of a change detection between classes of interest where values are greater than zero (not missing data and not cloud). Where Layer 3 is zero and the change array contains a value > 0, this date will be burned into the report layer 3. Where Layer 3 is non-zero it is set to the earlier date.

Layer 4: Combined Classifier + dNDVI Hybrid Change Detection Count: Count if a change was detected after a first change had already been detected previously.

Layer 5: Combined Classifier + dNDVI Validated No Change Detection Count: Count if no change was detected after a first change had previously been detected.

Layer 6: Cloud Occlusion Count: Count if a pixel is obscured by cloud or out-of-orbit after a first change had been detected.

Layer 7: Valid Pixel Count: Total number of valid (no cloud, no missing data) images covering this pixel since first change was detected.

Layer 8: Change Detection Repeatability: Repeatability of change detection after first change is detected - as a percentage of available valid images.

Layer 9: Binary time-series decision: Based on percentage_probability_threshold and minimum_required_validated_detections_threshold.

Layer 10: Binary time-series decision by first-change date: First change date masked by Binary Decision - Layer 9.

Layer 11: dNDVI Only Change Detection Count: Count if a change was detected by the dNDVI test and that was not cloud occluded (or out-of-orbit).

Layer 12: Binary time-series decision: Based on dNDVI Only and minimum_required_dNDVI_detections_threshold.

Layer 13: Binary time-series decision: Based on Machine Learning Classifier Only.

Layer 14: Combined Classifier+dNDVI Binary time-series decision: Based on machine learning classifier and dNDVI.

Layer 15: FROM Classification Count.

Layer 16: TO Classification Count.

Layer 17: Binary Decision Thresholds on FROM and TO Classification Counts.

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_change"]:
    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Creating change layers from stacked class images.")
    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Changes of interest:")
    tile_log.info(
        "  from any of the classes {}".format(config_dict["from_classes"])
    )
    tile_log.info("  to   any of the classes {}".format(config_dict["to_classes"]))

    # optionally sieve the class images
    if sieve > 0:
        tile_log.info("Applying sieve to classification outputs.")
        sieved_paths = raster_manipulation.sieve_directory(
            in_dir=categorised_image_dir,
            out_dir=sieved_image_dir,
            neighbours=8,
            sieve=sieve,
            out_type="GTiff",
            skip_existing=skip_existing,
        )
        # if sieve was chosen, work with the sieved class images
        class_image_dir = sieved_image_dir
    else:
        # if sieve was not chosen, work with the original class images
        class_image_dir = categorised_image_dir

    # get all image paths in the classification maps directory except the class composites
    class_image_paths = [
        f.path
        for f in os.scandir(class_image_dir)
        if f.is_file() and f.name.endswith(".tif") and not "composite_" in f.name
    ]
    if len(class_image_paths) == 0:
        raise FileNotFoundError(
            "No class images found in {}.".format(class_image_dir)
        )

    # sort class images by image acquisition date
    class_image_paths = list(
        filter(filesystem_utilities.get_image_acquisition_time, class_image_paths)
    )
    class_image_paths.sort(
        key=lambda x: filesystem_utilities.get_image_acquisition_time(x)
    )
    for index, image in enumerate(class_image_paths):
        tile_log.info("{}: {}".format(index, image))

    # find the latest available composite
    try:
        latest_composite_name = filesystem_utilities.sort_by_timestamp(
            [
                image_name
                for image_name in os.listdir(composite_dir)
                if image_name.endswith(".tif")
            ],
            recent_first=True,
        )[0]
        latest_composite_path = os.path.join(composite_dir, latest_composite_name)
        tile_log.info("Most recent composite at {}".format(latest_composite_path))
    except IndexError:
        tile_log.critical(
            "Latest composite not found. The first time you run this script, you need to include the "
            "--build-composite flag to create a base composite to work off. If you have already done this,"
            "check that the earliest dated image in your images/merged folder is later than the earliest"
            " dated image in your composite/ folder."
        )
        sys.exit(1)

    latest_class_composite_path = os.path.join(
        class_image_dir,
        [
            f.path
            for f in os.scandir(class_image_dir)
            if f.is_file()
            and os.path.basename(latest_composite_path)[:-4] in f.name
            and f.name.endswith(".tif")
        ][0],
    )

    tile_log.info(
        f"Most recent classification map of a composite at {latest_class_composite_path}"
    )
    if not os.path.exists(latest_class_composite_path):
        tile_log.critical(
            "Latest class composite not found. The first time you run this "+
            "script, you need to include the --build-composite flag to create "+
            "a base composite to work off. If you have already done this,"+
            " check that the earliest dated image in your images/merged folder "+
            "is later than the earliest dated image in your composite/ folder. "+
            "Then, you need to run the --classify option."
        )
        sys.exit(1)

    before_timestamp = filesystem_utilities.get_change_detection_dates(
        os.path.basename(latest_class_composite_path)
    )[0]
    # Timestamp report with the date of most recent classified image that contributes to it
    after_timestamp = filesystem_utilities.get_image_acquisition_time(
        os.path.basename(class_image_paths[-1])
    )
    output_product = os.path.join(
        report_image_dir,
        "report_{}_{}_{}.tif".format(
            before_timestamp.strftime("%Y%m%dT%H%M%S"),
            tile_to_process,
            after_timestamp.strftime("%Y%m%dT%H%M%S"),
        ),
    )
    tile_log.info("Report file name will be {}".format(output_product))

    # if a report file exists, archive it
    if os.path.isfile(output_product):
        tile_log.info(f"Report file already exists:  {output_product}")
        # if an earlier report file exists, archive it
        output_products_existing = [
            f.path
            for f in os.scandir(report_image_dir)
            if f.is_file()
            and f.name.startswith("report_")
            and f.name.endswith(".tif")
        ]
        n_report_files = len(output_products_existing)

        if n_report_files > 0:
            for output_product_existing in output_products_existing:
                # do not archive the current report image file
                if output_product_existing != output_product:
                    tile_log.info(
                        "Found existing earlier report image product: "+
                        f" {output_product_existing}"
                        )

                    output_product_existing_archived = os.path.join(
                        os.path.dirname(output_product_existing),
                        "archived_" + os.path.basename(output_product_existing),
                    )

                    i = 1
                    if len(output_product_existing_archived.split(".")) == 2:
                        beginning = output_product_existing_archived.split(".")[0]
                    else:
                        if len(output_product_existing_archived.split(".")) > 2:
                            beginning = ".".join(
                                output_product_existing_archived.split(".")[0:-1]
                                )
                    # instead of appending _1 indefinitely, isolate the _N part
                    #   of the filename and increase it by one
                    archive_path = beginning + f"_{i}." + output_product_existing_archived.split(".")[-1]
                    if os.path.exists(archive_path):
                        while os.path.exists(archive_path):
                            i = i + 1
                            archive_path = beginning + f"_{i}." + to_path.split(".")[-1]
                    output_product_existing_archived = archive_path

                    filesystem_utilities.move_and_rename_old_file(
                        output_product_existing,
                        output_product_existing_archived
                        )
    else:
        tile_log.info(f"Report file to be created:  {output_product}")

    # find change patterns in the stack of classification images
    for index, image in enumerate(class_image_paths):
        before_timestamp = filesystem_utilities.get_change_detection_dates(
            os.path.basename(latest_class_composite_path)
        )[0]
        after_timestamp = filesystem_utilities.get_image_acquisition_time(
            os.path.basename(image)
        )
        tile_log.info("-----------------------------------------------------------------------------")
        tile_log.info(
            "Processing classification image {} of {}: {}".format(
                index+1, len(class_image_paths), image
            )
        )
        tile_log.info("  early time stamp: {}".format(before_timestamp))
        tile_log.info("  late  time stamp: {}".format(after_timestamp))
        change_raster = os.path.join(
            probability_image_dir,
            "change_{}_{}_{}.tif".format(
                before_timestamp.strftime("%Y%m%dT%H%M%S"),
                tile_to_process,
                after_timestamp.strftime("%Y%m%dT%H%M%S"),
            ),
        )
        tile_log.info(
            "  Change raster file to be created: {}".format(change_raster)
        )

        dNDVI_raster = os.path.join(
            probability_image_dir,
            "dNDVI_{}_{}_{}.tif".format(
                before_timestamp.strftime("%Y%m%dT%H%M%S"),
                tile_to_process,
                after_timestamp.strftime("%Y%m%dT%H%M%S"),
            ),
        )
        tile_log.info(
            "  dNDVI raster file to be created: {}".format(dNDVI_raster)
        )

        NDVI_raster = os.path.join(
            probability_image_dir,
            "NDVI_{}_{}_{}.tif".format(
                before_timestamp.strftime("%Y%m%dT%H%M%S"),
                tile_to_process,
                after_timestamp.strftime("%Y%m%dT%H%M%S"),
            ),
        )
        tile_log.info(
            "  NDVI raster file of change image to be created: {}".format(
                NDVI_raster
            )
        )

        '''
        The change_from_class_maps() function looks for changes from class
        'change_from' in the composite to any of the 'change_to_classes'
        in the change images. Pixel values are the acquisition date of the
        detected change of interest or zero.
        Optionally, changes will be confirmed by thresholding a vegetation
        index calculated from two bands if the difference between
        the more recent date and the older date is below the confirmation
        threshold (e.g. NDVI < -0.2).
        '''

        tile_log.info("Update of the report image product based on change "+
                      "detection image.")
        raster_manipulation.change_from_class_maps(
            old_class_path=latest_class_composite_path,
            new_class_path=image,
            change_raster=change_raster,
            dNDVI_raster=dNDVI_raster,
            NDVI_raster=NDVI_raster,
            change_from=from_classes,
            change_to=to_classes,
            report_path=output_product,
            skip_existing=skip_existing,
            old_image_dir=composite_dir,
            new_image_dir=l2_masked_image_dir,
            viband1=4,
            viband2=3,
            dNDVI_threshold=-0.2,
            log=tile_log,
        )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Post-classification change detection complete.")
    tile_log.info("---------------------------------------------------------------")
    tile_log.info(
        "Report image product completed / updated: {}".format(output_product)
    )

## Vectorise the report image file
Now, we convert the report image file to vector format. This saves disk space.

In [ ]:
%%time
if config_dict["do_all"] or config_dict["do_vectorise"]:
    # get other parameters
    epsg = config_dict["epsg"]
    level_1_boundaries_path = config_dict["level_1_boundaries_path"]

    # get all report.tif file names that are within the report_image_dir
    search_pattern = os.path.join(report_image_dir, "report*.tif")
    change_report_paths = glob.glob(
        os.path.join(config_dict["tile_dir"], search_pattern)
    )
    if len(change_report_paths) == 0:
        tile_log.error("No change report image path(s) found that match pattern: \'"+
                       f"{search_pattern}\'")
        sys.exit()
    else:
        tile_log.info("Change report image path(s) found:")
        for p in change_report_paths:
            tile_log.info(f"  {p}")

    tile_log.info("--" * 20)
    tile_log.info("Starting Vectorisation of the Change Report Rasters")
    tile_log.info("--" * 20)

    # iterate over the list of report image files
    output_vector_files = []
    for change_report_path in change_report_paths:
        tile_log.info(f"Processing report image file: {change_report_path}")

        if os.path.exists(change_report_path[:-4]+".shp"):
            tile_log.info(f"Skipping. Found {change_report_path[:-4]}.shp")

        else:
            path_vectorised_binary = vectorisation.vectorise_from_band(
                change_report_path=change_report_path,
                band=9, # was 15 before but missed a lot of alerts in the vector file
                log=tile_log
            )

            path_vectorised_binary_filtered = vectorisation.clean_zero_nodata_vectorised_band(
                vectorised_band_path=path_vectorised_binary,
                log=tile_log
            )

            tile_log.info(f"vectorised_file_path = {path_vectorised_binary_filtered}")

            # Note: zonal_statistics() returns None if no statistics were computed.
            # In cases where the shapefile is not right, this causes an error.
            rb_ndetections_zstats_df = vectorisation.zonal_statistics(
                raster_path=change_report_path,
                shapefile_path=path_vectorised_binary_filtered,
                report_band=5,
                log=tile_log
                )

            rb_confidence_zstats_df = vectorisation.zonal_statistics(
                raster_path=change_report_path,
                shapefile_path=path_vectorised_binary_filtered,
                report_band=9,
                log=tile_log
            )

            rb_first_changedate_zstats_df = vectorisation.zonal_statistics(
                raster_path=change_report_path,
                shapefile_path=path_vectorised_binary_filtered,
                report_band=4,
                log=tile_log
            )

            # table joins, area, lat lon, county
            output_vector_files.append(
                vectorisation.merge_and_calculate_spatial(
                    rb_ndetections_zstats_df=rb_ndetections_zstats_df,
                    rb_confidence_zstats_df=rb_confidence_zstats_df,
                    rb_first_changedate_zstats_df=rb_first_changedate_zstats_df,
                    path_to_vectorised_binary_filtered=path_vectorised_binary_filtered,
                    write_csv=False,
                    write_shapefile=True,
                    write_kml=True,
                    write_pkl=False,
                    change_report_path=change_report_path,
                    log=tile_log,
                    epsg=epsg,
                    level_1_boundaries_path=level_1_boundaries_path,
                    tileid=tile_to_process,
                    delete_intermediates=True,
                )
            )
    tile_log.info("---------------------------------------------------------------")
    tile_log.info("Report image vectorised. Output file(s) created:")
    for i in range(len(output_vector_files)):
        tile_log.info(f"  {output_vector_files[i]}")
    tile_log.info("---------------------------------------------------------------")

## Final Housekeeping

Finally, we run some more housekeeping, deleting or compressing unnecessary files, depending on the argument supplied at the beginning of this session.

In [ ]:
%%time
directories = [
    categorised_image_dir,
    sieved_image_dir,
    probability_image_dir,
]
if config_dict["do_all"] or config_dict["do_change"]:
    for directory in directories:
        if os.path.exists(directory):
            tile_log.info("---------------------------------------------------------------")
            tile_log.info(f"Compressing tiff files in directory {directory} and "+
                          "all subdirectories")
            tile_log.info("---------------------------------------------------------------")
            for root, dirs, files in os.walk(directory):
                all_tiffs = [
                    image_name for image_name in files if image_name.endswith(".tif")
                ]
                for this_tiff in all_tiffs:
                    raster_manipulation.compress_tiff(
                        os.path.join(root, this_tiff),
                        os.path.join(root, this_tiff),
                        log=tile_log
                    )

            tile_log.info("---------------------------------------------------------------")

    if config_dict["do_delete"]:
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info(
            "Deleting intermediate class images used in change detection."
        )
        tile_log.info(
            "They can be recreated from the cloud-masked, band-stacked L2A images and the saved model."
        )
        tile_log.info(
            "---------------------------------------------------------------"
        )
        for directory in directories:
            if os.path.exists(directory):
                paths = [f for f in os.listdir(directory)]
                for f in paths:
                    # keep the classified composite layers and the report image
                    #    product for the next change detection
                    if not f.startswith("composite_") and not f.startswith("report_"):
                        tile_log.info(f"Deleting {os.path.join(directory, f)}")
                        if os.path.isdir(os.path.join(directory, f)):
                            shutil.rmtree(os.path.join(directory, f))
                        else:
                            os.remove(os.path.join(directory, f))
        tile_log.info(
            "---------------------------------------------------------------"
        )
        tile_log.info("Deletion of intermediate file products complete.")
        tile_log.info(
            "---------------------------------------------------------------"
        )
    else:
        if config_dict["do_zip"]:
            tile_log.info(
                "---------------------------------------------------------------"
            )
            tile_log.info(
                "Zipping intermediate class images used in change detection"
            )
            tile_log.info(
                "---------------------------------------------------------------"
            )
            directories = [categorised_image_dir, sieved_image_dir]
            for directory in directories:
                filesystem_utilities.zip_contents(
                    directory, notstartswith=["composite_", "report_"]
                )
            tile_log.info(
                "---------------------------------------------------------------"
            )
            tile_log.info("Zipping complete")
            tile_log.info(
                "---------------------------------------------------------------"
            )

    tile_log.info("---------------------------------------------------------------")
    tile_log.info(
        "Change detection and report image product updating, file compression, zipping"
    )
    tile_log.info(
        "and deletion of intermediate file products (if selected) are complete."
    )
    tile_log.info("---------------------------------------------------------------")

    if config_dict["do_delete"]:
        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Deleting temporary directories starting with 'tmp*'")
        tile_log.info("These can be left over from interrupted processing runs.")
        tile_log.info("---------------------------------------------------------------")
        directory = pyeo_dir
        for root, dirs, files in os.walk(directory):
            temp_dirs = [d for d in dirs if d.startswith("tmp")]
            for temp_dir in temp_dirs:
                tile_log.info(f"Deleting {os.path.join(root, temp_dir)}")
                if os.path.isdir(os.path.join(directory, f)):
                    shutil.rmtree(os.path.join(directory, f))
                else:
                    tile_log.warning(
                        "This should not have happened. "+
                        f"{os.path.join(root, temp_dir)} is not a directory. "+
                        "Skipping deletion."
                    )
        tile_log.info("---------------------------------------------------------------")
        tile_log.info("Deletion of temporary directories complete.")
        tile_log.info("---------------------------------------------------------------")


tile_log.info("---------------------------------------------------------------")
tile_log.info("---                  PROCESSING END                         ---")
tile_log.info("---------------------------------------------------------------")